In [4]:
from pyspark.sql import SparkSession
import os,tempfile,shutil

spark=(SparkSession.builder
    .appName("CSV Options")
    .master("local[*]")
    .getOrCreate())

spark.sparkContext.setLogLevel("Error")

#create a temp folder 

base_dir=tempfile.mkdtemp()
csv_path="./messy.csv"

data="""id|name|city|amount|comment
1|"Amit"|"Mumbai"|100.5|"Normal row
2|"Riya"|"New york, USA"|200.0|"City contains comma"
3|"John"|"Delhi"|NULL|"AMount id NULL string -> should become null "
4|"Sara"|"Kolakata"|300.75|"HE said \\"Hello\\" to me"
5|"Gaurav"|"Bangalore"|400.0|"This is multiline comments
second line of comments"
6|"Badrow"|"chennai"|500.0
7|"Extracol"|"pune"|600.0|"ok"|"extra"
"""

with open(csv_path,"w",encoding="utf-8") as f:
    f.write(data)
print("CSv created at:",csv_path)




CSv created at: ./messy.csv


In [3]:
import os 
print(os.getcwd())

/Users/anujshahdeo/Documents/GithubRepos/Spark_DE/Module_4


In [5]:
# read CSV 
df1=spark.read.format("csv").load(csv_path)

In [8]:
df1.show(truncate=False)

+--------------------------------------------------------------------+
|_c0                                                                 |
+--------------------------------------------------------------------+
|id|name|city|amount|comment                                         |
|1|"Amit"|"Mumbai"|100.5|"Normal row                                 |
|2|"Riya"|"New york                                                  |
|3|"John"|"Delhi"|NULL|"AMount id NULL string -> should become null "|
|4|"Sara"|"Kolakata"|300.75|"HE said \"Hello\" to me"                |
|5|"Gaurav"|"Bangalore"|400.0|"This is multiline comments            |
|second line of comments"                                            |
|6|"Badrow"|"chennai"|500.0                                          |
|7|"Extracol"|"pune"|600.0|"ok"|"extra"                              |
+--------------------------------------------------------------------+



In [9]:
df1.printSchema()

root
 |-- _c0: string (nullable = true)



    - columns will be c0,c1
    - everthing is string 
        - | won't be trated as seprator correctly (if expects comma by default)
    - Multiple row will break badly 

In [10]:
#Step 2 
# Read csv correct way 

df2= (spark.read
        .format("csv")
        .option("header","true") # columns 
        .option("sep","|") # delimieter
        .load(csv_path))

In [11]:
df2.show(truncate=False)

+------------------------+--------+-------------+------+--------------------------------------------+
|id                      |name    |city         |amount|comment                                     |
+------------------------+--------+-------------+------+--------------------------------------------+
|1                       |Amit    |Mumbai       |100.5 |Normal row                                  |
|2                       |Riya    |New york, USA|200.0 |City contains comma                         |
|3                       |John    |Delhi        |NULL  |AMount id NULL string -> should become null |
|4                       |Sara    |Kolakata     |300.75|HE said "Hello" to me                       |
|5                       |Gaurav  |Bangalore    |400.0 |This is multiline comments                  |
|second line of comments"|NULL    |NULL         |NULL  |NULL                                        |
|6                       |Badrow  |chennai      |500.0 |NULL                      

In [12]:
df2.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- comment: string (nullable = true)



In [19]:
#step 3 Enable inferschema (demo/ not prod)

df3= (spark.read
        .format("csv")
        .option("header","true") # columns 
        .option("sep","|") # delimieter
        .option("inferSchema","true") # automatically guess datatype 
        .load(csv_path))


In [20]:
df3.show(truncate=False)

+------------------------+--------+-------------+------+--------------------------------------------+
|id                      |name    |city         |amount|comment                                     |
+------------------------+--------+-------------+------+--------------------------------------------+
|1                       |Amit    |Mumbai       |100.5 |Normal row                                  |
|2                       |Riya    |New york, USA|200.0 |City contains comma                         |
|3                       |John    |Delhi        |NULL  |AMount id NULL string -> should become null |
|4                       |Sara    |Kolakata     |300.75|HE said "Hello" to me                       |
|5                       |Gaurav  |Bangalore    |400.0 |This is multiline comments                  |
|second line of comments"|NULL    |NULL         |NULL  |NULL                                        |
|6                       |Badrow  |chennai      |500.0 |NULL                      

In [21]:
df3.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- comment: string (nullable = true)



In [22]:
#Step 4 Use nullvalue so NULL become actual null

df4= (spark.read
        .format("csv")
        .option("header","true") # columns 
        .option("sep","|") # delimieter
        .option("inferSchema","true") # automatically guess datatype 
        .option("nullValue","NULL")
        .load(csv_path))

In [23]:
df4.show(truncate=False)

+------------------------+--------+-------------+------+--------------------------------------------+
|id                      |name    |city         |amount|comment                                     |
+------------------------+--------+-------------+------+--------------------------------------------+
|1                       |Amit    |Mumbai       |100.5 |Normal row                                  |
|2                       |Riya    |New york, USA|200.0 |City contains comma                         |
|3                       |John    |Delhi        |NULL  |AMount id NULL string -> should become null |
|4                       |Sara    |Kolakata     |300.75|HE said "Hello" to me                       |
|5                       |Gaurav  |Bangalore    |400.0 |This is multiline comments                  |
|second line of comments"|NULL    |NULL         |NULL  |NULL                                        |
|6                       |Badrow  |chennai      |500.0 |NULL                      

In [24]:
df4.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- comment: string (nullable = true)



In [26]:
df5= (spark.read
        .format("csv")
        .option("header","true") # columns 
        .option("sep","|") # delimieter
        .option("inferSchema","true") # automatically guess datatype 
        .option("nullValue","NULL")
        .option("quote",'"')
        .option("escape","\\")
        .option("multiLine","True")
        .load(csv_path))

In [28]:
df5.show(truncate=False)

+---+--------+---------+------+---------------------------------------------------+
|id |name    |city     |amount|comment                                            |
+---+--------+---------+------+---------------------------------------------------+
|1  |Amit    |Mumbai   |100.5 |"Normal row\n2|"Riya"                              |
|3  |John    |Delhi    |NULL  |AMount id NULL string -> should become null        |
|4  |Sara    |Kolakata |300.75|HE said "Hello" to me                              |
|5  |Gaurav  |Bangalore|400.0 |This is multiline comments\nsecond line of comments|
|6  |Badrow  |chennai  |500.0 |NULL                                               |
|7  |Extracol|pune     |600.0 |ok                                                 |
+---+--------+---------+------+---------------------------------------------------+



In [32]:
df6= (spark.read
        .format("csv")
        .option("header","true") # columns 
        .option("sep","|") # delimieter
        .option("inferSchema","true") # automatically guess datatype 
        .option("nullValue","NULL")
        .option("quote",'"')
        .option("escape","\\")
        .option("multiLine","True")
        .option("mode","PERMISSIVE")
        .option("columnNameOfCorruptRecord","_badrecords")
        .load(csv_path))

In [33]:
df6.show(truncate=False)

+---+--------+---------+------+---------------------------------------------------+
|id |name    |city     |amount|comment                                            |
+---+--------+---------+------+---------------------------------------------------+
|1  |Amit    |Mumbai   |100.5 |"Normal row\n2|"Riya"                              |
|3  |John    |Delhi    |NULL  |AMount id NULL string -> should become null        |
|4  |Sara    |Kolakata |300.75|HE said "Hello" to me                              |
|5  |Gaurav  |Bangalore|400.0 |This is multiline comments\nsecond line of comments|
|6  |Badrow  |chennai  |500.0 |NULL                                               |
|7  |Extracol|pune     |600.0 |ok                                                 |
+---+--------+---------+------+---------------------------------------------------+



In [34]:
df6.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- comment: string (nullable = true)



In [36]:
df6.filter(df6["_badrecords"].isNotNull()).show(truncate=False)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `_badrecords` cannot be resolved. Did you mean one of the following? [`id`, `name`, `city`, `amount`, `comment`].